In [964]:
from z3 import *
import numpy as np
class f2Poly:
    def __init__(self, terms=None):
        self.terms = set() if terms is None else terms
    def __add__(self, other):
        return f2Poly(self.terms ^ other.terms)
    def __mul__(self, other):
        terms1 = self.terms
        # 情况1：other 是数字（int）
        if isinstance(other, int):
            scalar = other % 2  # 模 2 到 F₂
            if scalar == 0:
                return f2Poly(set())  # 0 * p = 0
            else:
                return self           # 1 * p = p
        # 情况2：other 是 f2Poly
        terms2 = other.terms
        if not terms1 or not terms2:
            return f2Poly(set())  # 0 * anything = 0
        result = set()
        _add = result.add
        _remove = result.remove
        _in = result.__contains__
        for (i1, j1) in terms1:
            for (i2, j2) in terms2:
                term = (i1 + i2, j1 + j2)
                if _in(term):
                    _remove(term)
                else:
                    _add(term)
        return f2Poly(result)
    def __pow__(self, n):
        if n == 0: return f2Poly({(0, 0)})
        if len(self.terms) == 0:  # 零多项式
            if n > 0: return f2Poly(set())
            else: raise ValueError("0 的负幂次无定义")
        if len(self.terms) == 1:  # 单项式
            (i, j) = next(iter(self.terms))
            return f2Poly({(i * n, j * n)})
        # 多项式：快速幂
        if n < 0: raise ValueError("不可逆")
        elif n == 1: return self
        elif n % 2 == 0: return (self * self) ** (n // 2)
        else: return self * (self ** (n - 1))
    def deg(self):
        x_powers = set()
        y_powers = set()
        if self.terms==set():
            x_powers.add(0)
            y_powers.add(0)
        else:
            for i, j in self.terms:
                x_powers.add(i)
                y_powers.add(j)
        return x_powers, y_powers
    def __repr__(self):
        if not self.terms:
            return "0"
        return " + ".join(
            f"x^{i}y^{j}" if i and j else
            f"x^{i}" if i else
            f"y^{j}" if j else
            "1"
            for i, j in self.terms
        )
def antimap(poly):
    inverted_terms = {( -i, -j ) for (i, j) in poly.terms}
    return f2Poly(inverted_terms)
def commute(a, b):
    # 预提取输入
    a0, a1, a2, a3 = a[0][0], a[1][0], a[2][0], a[3][0]
    b0, b1, b2, b3 = b[0][0], b[1][0], b[2][0], b[3][0]
    # 局部引用加速
    _antimap = antimap
    _f2Poly = f2Poly
    result = _f2Poly()
    result += _antimap(a0) * b2
    result += _antimap(a1) * b3
    result += _antimap(a2) * b0
    result += _antimap(a3) * b1
    return result
def excimap(*A, oper):
    result = []
    for a in A:
        excitation = commute(a, oper)
        result.append(excitation)
    return result  # 返回 [F2Poly, F2Poly, ...]，就是一个行向量
def tdmap(f, m):
    shifts = [(i, j) for j in range(-m, m + 1) for i in range(-m, m + 1)]
    f_terms_list = [poly.terms for poly in f]
    result = [None] * len(shifts)
    for idx, (i, j) in enumerate(shifts):
        shifted_f = []
        for terms in f_terms_list:
            shifted_terms = {(i + di, j + dj) for (di, dj) in terms}
            shifted_f.append(f2Poly(shifted_terms))
        result[idx] = shifted_f
    return result
def trunmap(f,k):
    ff=np.zeros((2*k+1)**2,dtype=int)
    for (i, j) in f.terms:
        ff[i+k+(j+k)*(2*k+1)]=1
    return ff
def kdeg(f,m):
    a=[]
    b=[]
    for i in range(len(tdmap(f,m)[0])):
        if tdmap(f,m)[0][i]!=f2Poly() and tdmap(f,m)[0][i]!=f2Poly({(0,0)}):
            a.append(max(max(max(tdmap(f,m)[0][i].deg()[0]),-min(tdmap(f,m)[0][i].deg()[0])),
                     max(max(tdmap(f,m)[0][i].deg()[1]),-min(tdmap(f,m)[0][i].deg()[1]))))
    for i in range(len(tdmap(f,m)[-1])):
        if tdmap(f,m)[-1][i]!=f2Poly() and tdmap(f,m)[-1][i]!=f2Poly({(0,0)}):
            b.append(max(max(max(tdmap(f,m)[-1][i].deg()[0]),-min(tdmap(f,m)[-1][i].deg()[0])),
                     max(max(tdmap(f,m)[-1][i].deg()[1]),-min(tdmap(f,m)[-1][i].deg()[1]))))
    return max(max(a),max(b))
def potonu(f,k):
    a=[]
    for i in range(len(f)):
        b=[]
        for j in range(len(f[i])):
            b.extend(trunmap(f[i][j],k))
        a.append(b)
    return a
def invtrunmap(f,k):
    ff=f2Poly()
    for i in range(len(f)):
        if f[i]!=0:
            a=i%(2*k+1)-k
            b=i//(2*k+1)-k
            ff=ff+f2Poly({(a,b)})
    return ff
def m1(*A,m):
    m1=[]
    m1.extend(tdmap(excimap(*A,oper=x1),m))
    m1.extend(tdmap(excimap(*A,oper=x2),m))
    m1.extend(tdmap(excimap(*A,oper=z1),m))
    m1.extend(tdmap(excimap(*A,oper=z2),m))
    k=max(max(max(kdeg(excimap(s1,s2,oper=x1),m),kdeg(excimap(s1,s2,oper=x2),m)),kdeg(excimap(s1,s2,oper=z1),m)),kdeg(excimap(s1,s2,oper=z2),m))
    a=[]
    for i in range(len(m1)):
        b=[]
        for j in range(len(m1[i])):
            b.extend(trunmap(m1[i][j],k))
        a.append(b)
    return np.array(a)
def mod2(matrix):
    return np.mod(matrix, 2)
def gf2_rref(matrix):
    matrix = mod2(matrix)
    rows, cols = matrix.shape
    row, col = 0, 0
    while row < rows and col < cols:
        if matrix[row, col] == 0:
            for r in range(row + 1, rows):
                if matrix[r, col] == 1:
                    matrix[[row, r]] = matrix[[r, row]]
                    break
        if matrix[row, col] == 1:
            for r in range(rows):
                if r != row and matrix[r, col] == 1:
                    matrix[r] = mod2(matrix[r] + matrix[row])
            row += 1
        col += 1
    return matrix
def m2(*A,m):
    stabdag=[]
    for a in A:
        stabdag.append(np.array(a).T[0])
    M2=[]
    for i in range(len(stabdag)):
        M2.append(tdmap(stabdag[i],m))
    k=kdeg(stabdag[0],m)
    for i in range(1,len(stabdag)):
        k=max(kdeg(stabdag[i-1],m),kdeg(stabdag[i],m))
    a=[]
    for i in range(len(stabdag)):
        b=[]
        for j in range(len(stabdag[i])):
            b.extend(trunmap(stabdag[i][j],k))
        a.append(b)
    return np.array(a)
x1=[[f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()],
    [f2Poly()]]
x2=[[f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()]]
z1=[[f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()]]
z2=[[f2Poly()],
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})]]
# 示例 1: 3x + 2y → 模 2 后是 x
s1=[
    [f2Poly({(0, 0): 1,(-1,0):1})],
    [f2Poly({(0, 0): 1,(0,-1):1})],
    [f2Poly()],
    [f2Poly()]
]
s2=[
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0, 0): 1,(0,1):1})],
    [f2Poly({(0, 0): 1,(1,0):1})],
]
f=f2Poly({(0,0)})
print(m2(s1,s2,m=3).shape)

(2, 324)


In [959]:
print(np.array(s1).T[0][1])

1 + y^-1
